In [16]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
import base64


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animal_shelter import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacuser"
password = "123"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object and accept projection json input
df = pd.DataFrame.from_records(shelter.read({}))

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso_Salvare_Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.A(href="https://www.snhu.edu", children=html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),style={'width': '100px', 'height': '100px'})),
    html.Center(html.B(html.H1('Glenn Bacon 4/18/2021 SNHU CS-340 Dashboard'))),
    html.Hr(),
    html.Div(className='row',
            style={'display' : 'flex'},
                children=[dcc.RadioItems(id='radio-button',options=[
                    {"label": "Water Rescue", "value": "Water Rescue"},
                    {"label": "Wilderness Rescue", "value": "Wilderness Rescue"},
                    {"label": "Disaster Rescue", "value": "Disaster Rescue"},
                    {"label": "Reset", "value": "Reset"}])
                ]),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        #FIXME: Set up the features for your interactive data table to make it user-friendly for your client
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable=False,
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current=0,
        page_size=10,
    ),
    html.Br(),
    html.Hr(),
    #This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
             style={'display' : 'flex'},
             children=[
                 html.Div(
                     id='graph-id',
                     className='col s12 m6',
                 ),
                 html.Div(
                     id='map-id',
                     className='col s12 m6',
                 )
             ])
])

#############################################
# Interaction Between Components / Controller
#############################################
#This callback will highlight a row on the data table when the user selects it

@app.callback(
    Output('datatable-id', 'data'),
    [Input('radio-button', 'value')])
def on_click(radio_value):
    # start case
    if radio_value == "Water Rescue":
        df = pd.DataFrame(list(shelter.read({"outcome":"Water"})))
    elif radio_value == "Wilderness Rescue":
        df = pd.DataFrame(list(shelter.read({"outcome":"Wilderness Rescue"})))
    elif radio_value == "Disaster Rescue":
        df = pd.DataFrame(list(shelter.read({"outcome":"Disaster Rescue"})))
    elif radio_value == "Reset":
        df = pd.DataFrame(list(shelter.read({})))
    return df.to_dict('records')

@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) #
    dff = pd.DataFrame.from_dict(viewData)
    breeds = []
    for i in range(0, 10):
        breeds.append(str(dff.iloc[i,5]))
    return [
        dcc.Graph(
            figure={
                'data':[
                    {
                        'labels':breeds,
                        'type':'pie'
                    }
                ],
                'layout':{
                    'title':'Breeds',
                }
            }
        )  
    ]
    
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
#FIXME Add in the code for your geolocation chart
    dff = pd.DataFrame.from_dict(viewData)
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[dff.iloc[0,14],dff.iloc[0,15]], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[dff.iloc[0,14],dff.iloc[0,15]], children=[
                dl.Tooltip(dff.iloc[0,4]),
                dl.Popup([
                    html.H1(dff.iloc[0,10]),
                    html.P(dff.iloc[0,9])
                ])
            ])
        ])
    ]
app
